# Google Drive File Organizer
This notebook organizes files in your Google Drive and suggests folders where files should be moved based on their names and types. It uses the Google Drive API to list files and folders and the OpenAI API to classify them.

## Setup and Installation
In this section, we'll install necessary libraries and authenticate with Google and OpenAI.

In [1]:
# Install required packages
!pip install openai --quiet

import os
import pandas as pd
from google.colab import auth, drive, userdata
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from openai import OpenAI
from tqdm.notebook import tqdm
from google.auth import default

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [2]:
# Authenticate and create the required clients
auth.authenticate_user()
drive.mount('/content/drive')
creds, _ = default()

# Initialize OpenAI client
client = OpenAI(api_key=userdata.get('openai_key'))

Mounted at /content/drive


## Google Drive API Functions
The following functions interact with the Google Drive API to fetch files and folders.

In [3]:
# Google Drive API Service Initialization
def get_drive_service():
    """Initialize and return the Google Drive service."""
    return build('drive', 'v3', credentials=creds)

In [4]:
# Fetching Files and Folders from Google Drive
def get_personal_files(service):
    """Retrieve a list of personal files from Google Drive."""
    files = []
    page_token = None
    query = "'me' in owners and mimeType != 'application/vnd.google-apps.folder'"

    while True:
        results = service.files().list(
            q=query,
            pageSize=1000,
            fields="nextPageToken, files(id, name, mimeType, parents)",
            pageToken=page_token
        ).execute()

        files.extend(results.get('files', []))
        page_token = results.get('nextPageToken')

        if not page_token:
            break

    return files

In [5]:
def get_shared_drives(service):
    """Retrieve a list of shared drives."""
    drives = []
    page_token = None

    while True:
        response = service.drives().list(pageSize=100, pageToken=page_token).execute()
        drives.extend(response.get('drives', []))
        page_token = response.get('nextPageToken')

        if not page_token:
            break

    return drives

In [6]:
def get_folders_in_drive(service, drive_id):
    """Retrieve a list of folders in a specific shared drive."""
    folders = []
    page_token = None
    query = "mimeType='application/vnd.google-apps.folder'"

    while True:
        results = service.files().list(
            q=query,
            corpora='drive',
            driveId=drive_id,
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            fields="nextPageToken, files(id, name, parents)",
            pageToken=page_token
        ).execute()

        folders.extend(results.get('files', []))
        page_token = results.get('nextPageToken')

        if not page_token:
            break

    return folders

In [14]:
def get_folder_path(service, folder_id):
    """Recursively build the full path of the folder using its ID."""
    path = []
    while folder_id:
        try:
            folder = service.files().get(
                fileId=folder_id,
                fields="id, name, parents",
                supportsAllDrives=True  # Include this for shared drives
            ).execute()
            path.append(folder['name'])
            folder_id = folder.get('parents', [None])[0]
        except HttpError as error:
            print(f"Error fetching folder path for ID {folder_id}: {error}")
            return None  # Return None if the folder cannot be found
    return '/'.join(reversed(path)) if path else None

## OpenAI API Function for Classification
This function uses the OpenAI API to classify files and suggest folders where they should be moved.

In [15]:
def classify_file(file_name, file_type, folders):
    """Classify a file and suggest whether it should be moved and to which folder."""
    prompt = f"""Given the file name '{file_name}' (type: {file_type}) and the following list of folders:
{', '.join([folder['name'] for folder in folders])}

Should this file be moved? If yes, to which folder? If no, say 'No move needed.'
Provide a brief explanation for your decision."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that organizes files."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7
        )

        content = response.choices[0].message.content.strip()

        if "No move needed" in content:
            return {
                "move_needed": False,
                "target_folder": None,
                "explanation": content,
                "folder_path": None,
                "folder_id": None
            }
        else:
            # Extract the folder name from the response if provided
            target_folder = next((folder for folder in folders if folder['name'] in content), None)

            if target_folder:
                print(f"Fetching path for folder ID: {target_folder['id']}")
                folder_path = get_folder_path(service, target_folder['id'])
                return {
                    "move_needed": True,
                    "target_folder": target_folder['name'],
                    "explanation": content,
                    "folder_path": folder_path,
                    "folder_id": target_folder['id']
                }
            else:
                print("Target folder not found in response content.")
                return {
                    "move_needed": True,
                    "target_folder": None,
                    "explanation": content,
                    "folder_path": None,
                    "folder_id": None
                }

    except HttpError as e:
        print(f"HTTP error during classification: {e}")
        return {
            "move_needed": False,
            "target_folder": None,
            "explanation": "Classification failed due to an HTTP error. Please try again later.",
            "folder_path": None,
            "folder_id": None
        }
    except Exception as e:
        print(f"Error during classification: {e}")
        return {
            "move_needed": False,
            "target_folder": None,
            "explanation": "Classification failed due to an error. Please try again later.",
            "folder_path": None,
            "folder_id": None
        }


## Running the Workflow
Now, let's run the functions to fetch the files and folders, classify them, and save the results.

In [9]:
# Initialize the Google Drive service
service = get_drive_service()

# Fetch personal files
print("Fetching personal files...")
personal_files = get_personal_files(service)

Fetching personal files...


In [10]:
# Fetch shared drives and folders
print("Fetching shared drives and folders...")
shared_drives = get_shared_drives(service)
all_folders = []
for drive in shared_drives:
    folders = get_folders_in_drive(service, drive['id'])
    all_folders.extend(folders)

Fetching shared drives and folders...


In [ ]:
# Classify files
print("Classifying files...")
data = []
for file in tqdm(personal_files[:10]):  # Adjust the slice for more files
    suggestion = classify_file(file['name'], file['mimeType'], all_folders)
    data.append({
        'File Name': file['name'],
        'Current Location': 'Personal Drive',
        'Move Needed': suggestion['move_needed'],
        'Target Folder': suggestion['target_folder'],
        'Explanation': suggestion['explanation'],
        'Folder Path': suggestion['folder_path'],
        'Folder ID': suggestion['folder_id']
    })

df = pd.DataFrame(data)

# Display the classification results
print("\nClassification Results:")
display(df)

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)